## **Final Code for product recommendation**

In [1]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':'17mly50Hi_6TQkxPvrCEfGY11_zf0t7jy'}) 
downloaded.GetContentFile('top10_products.pkl') 

downloaded = drive.CreateFile({'id':'1RvQcY4rFhtVgWkPcVAlqNPESouDfbFNQ'}) 
downloaded.GetContentFile('user_last_purchase.pkl') 

downloaded = drive.CreateFile({'id':'1zc1KeGkL70pXJi60l4D4Z9rIn1KP3gLy'}) 
downloaded.GetContentFile('day_reorder_rate.pkl') 

downloaded = drive.CreateFile({'id':'1m4EupTP1ynqEZPr6yoQQzCEcvwP4J4F9'}) 
downloaded.GetContentFile('p_days_since_prior_order_reorder_rate.pkl') 

downloaded = drive.CreateFile({'id':'1sD2C16knsFhQSEwywi2xWBkw642bcVOf'}) 
downloaded.GetContentFile('u_days_since_prior_order_reorder_rate.pkl') 

downloaded = drive.CreateFile({'id':'1gLRTtO_AeQ6B6gacjS5gGeLl-Dbit5nI'}) 
downloaded.GetContentFile('days_since_prior_reorder_rate.pkl') 

downloaded = drive.CreateFile({'id':'1sKYaYlHyGI35-vfBh9UjjSjYoM8eW2th'}) 
downloaded.GetContentFile('catboost_v3.pkl') 

downloaded = drive.CreateFile({'id':'1E-eAR92nV5dq27UhglfUcoIjug2YPsf8'}) 
downloaded.GetContentFile('product_features_v6.pkl') 

downloaded = drive.CreateFile({'id':'1SS2SGQshSQcXK2TeWzKj-K58cLCPvd17'}) 
downloaded.GetContentFile('user_features_v6.pkl') 

downloaded = drive.CreateFile({'id':'1T-RTnqaO5ICwSuHB1WDQcB_7WEIzbZHs'}) 
downloaded.GetContentFile('merged_user_product_features.pkl') 

downloaded = drive.CreateFile({'id':'1-fYKH5QvOcuiJ8IOd71eY43qNSKbuqSV'}) 
downloaded.GetContentFile('f1optimization_faron.py')

downloaded = drive.CreateFile({'id':'14aBOKGIZVwWBoWM43JZiW5tRaXz97zK6'}) 
downloaded.GetContentFile('hour_reorder_rate.pkl')

downloaded = drive.CreateFile({'id':'1fRFHq4fh-ClyrcKOctYJcc7EENRgn8Pa'}) 
downloaded.GetContentFile('product_mappings.pkl')

downloaded = drive.CreateFile({'id':'1JKqY3mqBKo7q7l3GtU8mo82ceNesQIPM'}) 
downloaded.GetContentFile('Data_v2.h5')

In [4]:
#loading libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from f1optimization_faron import get_best_prediction

In [5]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.1 MB/s 


In [6]:
catboost_v3=pd.read_pickle("catboost_v3.pkl")

## **Final function to return product recommendation**

In [7]:
ulp = pd.read_pickle("user_last_purchase.pkl")
top= pd.read_pickle('top10_products.pkl')
hour_rate = pd.read_pickle("hour_reorder_rate.pkl")
day_rate = pd.read_pickle("day_reorder_rate.pkl")
p_days_rate = pd.read_pickle("p_days_since_prior_order_reorder_rate.pkl")
u_days_rate = pd.read_pickle("u_days_since_prior_order_reorder_rate.pkl")
up_days_rate = pd.read_pickle("days_since_prior_reorder_rate.pkl")
merged_up_features = pd.read_pickle("merged_user_product_features.pkl")
products_x = pd.read_pickle("product_mappings.pkl")
product_features=pd.read_pickle("product_features_v6.pkl")
user_features=pd.read_pickle("user_features_v6.pkl")

In [8]:
product_features.head()

,product_id,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate
0,1,0.614627,5.845954,1.0,0.0,0.0,61,19,0.549341,0.043527
1,2,0.138298,10.138298,0.0,0.0,0.0,104,13,0.152933,0.032061
2,3,0.738516,6.374558,0.0,0.0,1.0,94,7,0.527619,0.000936
3,4,0.458689,9.472934,0.0,1.0,0.0,38,1,0.558656,0.006066
4,5,0.625000,6.375000,0.0,0.0,0.0,5,13,0.281312,0.032061


In [9]:
merged_up_features.head()

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak
0,1,196,0.157143,0.909091,1.363636,0,7
1,1,10258,0.142857,0.900000,3.600000,0,6
2,1,10326,0.014286,0.000000,5.000000,6,0
3,1,12427,0.142857,0.900000,3.300000,1,6
4,1,13032,0.057143,0.750000,6.500000,0,2


In [10]:

user_features.head()

,user_id,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio
0,1,0.728571,19,70,6.363636,17.272727,0.631579
1,2,0.464602,121,226,15.066667,15.200000,0.347107
2,3,0.625000,33,88,7.333333,11.083333,0.575758
3,4,0.055556,17,18,3.600000,11.000000,0.058824
4,5,0.391304,28,46,9.200000,9.200000,0.357143


In [11]:
def final(X = None):
    
    """
    Function returns the predicted product recommendations if its existing user, else returns 
    most frequently purchased product at that hour and on that day.
    
    user_id           : provided
    order_hour_of_day : calculated by seeing current time
    order_dow         : calulated by seeing current day of week
    
    """

    #get current time
    start_time = datetime.now()

    # datetime object containing current date and time
    now = datetime.now()

    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    today = int(dt_string.split("/")[0])

    #get data from user end
    user_id = int(X['user_id']) #user_id
    order_hour_of_day = int(dt_string.split(" ")[1].split(":")[0]) #current date
    order_dow = datetime.today().weekday() #current day of week

    """
    check if the user is new or existing,
    this file contains user's last order date
    
  
    """
    
    ulp = pd.read_pickle("user_last_purchase.pkl")
    
    #if user is new -> cold start -> recommend products which are most frequently purchased based on time and day of week
    if user_id not in ulp['user_id'].values:

        #get top 10 products based on hour of day and day of week
        top= pd.read_pickle('top10_products.pkl')
        top_products = top[(top['order_dow']==order_dow) & (top['order_hour_of_day']==order_hour_of_day)]['product_name'].values.tolist()
        top_products = {i: value for i,value in enumerate(top_products)}
        #paths = get_image_paths(top5_products)
        predictions={}
        predictions['top'] =  top_products

        # del ulp, top,now, today, dt_string, order_dow, order_hour_of_day

        end_time = datetime.now()
        difference = end_time - start_time
        #print("Total Time : {} seconds".format(difference))
        time = "{}".format(difference)
        
        return predictions,time
    
    #if user is existing one
    user_last_order_date = ulp[ulp['user_id']==user_id]['date'].values.tolist()[0]

    days_since_prior_order = today - int(user_last_order_date.split('-')[-1])
    # del ulp, now, today, dt_string, user_last_order_date
    
    #build features
    hour_rate = pd.read_pickle("hour_reorder_rate.pkl")
    day_rate = pd.read_pickle("day_reorder_rate.pkl")
    p_days_rate = pd.read_pickle("p_days_since_prior_order_reorder_rate.pkl")
    u_days_rate = pd.read_pickle("u_days_since_prior_order_reorder_rate.pkl")
    up_days_rate = pd.read_pickle("days_since_prior_reorder_rate.pkl")
    merged_up_features = pd.read_pickle("merged_user_product_features.pkl")

    #collect features based on hour of day , day of week, and user id
    featurized_data = merged_up_features[merged_up_features['user_id']==user_id]
    hour_r = hour_rate[hour_rate['order_hour_of_day']==order_hour_of_day]
    day_r = day_rate[day_rate['order_dow'] == order_dow]
    p_days = p_days_rate[p_days_rate['days_since_prior_order']==days_since_prior_order]
    u_days = u_days_rate[(u_days_rate['user_id']==user_id) & (u_days_rate['days_since_prior_order']==days_since_prior_order)]
    
    """
    if user never made a purchase in the gap of (days_since_prior_order) between 2 orders, make a new dataframe, 
    with the given number of days , and with reorder_rate = 0.0
    """
    
    if p_days.empty:
        #handle
        p_days = pd.DataFrame(columns = p_days.columns)
        products_x = pd.read_pickle('product_mappings.pkl')
        p_days['product_id'] = products_x['product_id']
        p_days['days_since_prior_order'] = days_since_prior_order
        p_days['p_days_since_prior_order_reorder_rate']=0.0
    
    up_days = up_days_rate[(up_days_rate['user_id']==user_id) & (up_days_rate['days_since_prior_order']==days_since_prior_order)]

    if up_days.empty:
        #handle
        up_days = pd.DataFrame(columns = up_days_rate.columns)
        products_x = pd.read_pickle('product_mappings.pkl')
        up_days['product_id'] = products_x['product_id']
        up_days['user_id'] = user_id
        up_days['days_since_prior_order'] = days_since_prior_order
        up_days['days_since_prior_reorder_rate']=0
        # del products_x
    
    if u_days.empty:
        #handle
        u_days = pd.DataFrame(columns = u_days.columns)
        df2 = {'user_id': user_id, 'days_since_prior_order': days_since_prior_order, 'u_days_since_prior_order_reorder_rate': 0}
        u_days = u_days.append(df2, ignore_index = True)
        # del df2

    up_days = up_days_rate[(up_days_rate['user_id']==user_id) & (up_days_rate['days_since_prior_order']==days_since_prior_order)]
    
    """
    if user never made a purchase in the gap of (days_since_prior_order) between 2 orders, make a new dataframe, 
    with the given number of days , and with reorder_rate = 0.0
    """
    if up_days.empty:
        #handle
        up_days = pd.DataFrame(columns = up_days_rate.columns)
        products_x = pd.read_pickle('product_mappings.pkl')
        up_days['product_id'] = products_x['product_id']
        up_days['user_id'] = user_id
        up_days['days_since_prior_order'] = days_since_prior_order
        up_days['days_since_prior_reorder_rate']=0
        # del products_x


    #print(up_days_rate[up_days_rate['user_id']==user_id])
    #print(u_days_rate[u_days_rate['user_id']==user_id])
    #print(day_rate)
    # del merged_up_features, hour_rate, day_rate, p_days_rate, u_days_rate, up_days_rate

    featurized_data = pd.merge(featurized_data, up_days, on = ['user_id', 'product_id'])

    featurized_data = pd.merge(featurized_data, hour_r, on = 'product_id')
    featurized_data = pd.merge(featurized_data, day_r, on = 'product_id')
    featurized_data = pd.merge(featurized_data, p_days, on = ['product_id','days_since_prior_order'])
    featurized_data = pd.merge(featurized_data, u_days, on = ['user_id','days_since_prior_order'])
    featurized_data = pd.merge(featurized_data, product_features, on = ['product_id'])
    featurized_data = pd.merge(featurized_data, user_features, on = ['user_id'])
    featurized_data = featurized_data[['user_id', 'product_id', 'u_p_order_rate', 'u_p_reorder_rate',\
                                       'u_p_avg_position', 'u_p_orders_since_last', 'max_streak', 'user_reorder_rate',\
                                       'user_unique_products', 'user_total_products', 'user_avg_cart_size', \
                                       'user_avg_days_between_orders', 'user_reordered_products_ratio', \
                                       'product_reorder_rate', 'avg_pos_incart', 'p_reduced_feat_1', 'p_reduced_feat_2', \
                                       'p_reduced_feat_3', 'aisle_id', 'department_id', 'aisle_reorder_rate', \
                                       'dept_reorder_rate', 'order_dow', 'order_hour_of_day', 'days_since_prior_order',\
                                       'hour_reorder_rate', 'day_reorder_rate', 'p_days_since_prior_order_reorder_rate',\
                                       'u_days_since_prior_order_reorder_rate', 'days_since_prior_reorder_rate']]

    # del up_days,u_days,p_days,day_r,hour_r

    #model
    with open("catboost_v3.pkl", "rb") as f:
        model = pickle.load(f)
    #model = pickle.load(open("catboost_v3.pkl", "rb"))
    
    data = featurized_data.drop(['user_id', 'product_id'], axis = 1)
    ypred = model.predict_proba(data)
    ypred = ypred[:,-1] #get probabilities of class 1
    # del data,model

    #run faron's optimization code to get most probable set of products which might be reordered
    
    #set showThreshold = True , while debugging to print the threshold
    recommended_products = get_best_prediction(featurized_data['product_id'].tolist(), ypred.tolist(), None, showThreshold = True)
    recommended_products = recommended_products.replace("None", "")
    recommended_products = list(map(int, recommended_products.split()))
    products_x = pd.read_pickle('product_mappings.pkl')
    recommended_products = products_x.loc[products_x['product_id'].isin(recommended_products)]['product_name'].values.tolist()
    recommended_products = {i: value for i,value in enumerate(recommended_products)}

    predictions= {}
    predictions['recommend'] = recommended_products

    end_time = datetime.now()
    difference = end_time - start_time
    #print("Total Time : {} seconds".format(difference))
    time = "{}".format(difference)

    # del featurized_data, products_x
    return predictions, time

In [12]:
#existing user

X = {}
X['user_id'] = 224
recommended_products = final(X)

print()
print("="*20)
print("Recommended products")
print("="*20)
for _,value in recommended_products[0]['recommend'].items():
    print(value)

Threshold : P(X) > 1.0000
Maximum F1 : 1.0000

Recommended products
Organic Green Onions
Russet Potato
Macaroni Elbows
Organic Half Tea & Half Lemonade
Organic Red On the Vine Tomato


In [19]:
#existing user

X = {}
X['user_id'] = 10005
recommended_products = final(X)

print()
print("="*20)
print("Recommended products")
print("="*20)
for _,value in recommended_products[0]['recommend'].items():
    print(value)

Threshold : P(X) > 1.0000
Maximum F1 : 1.0000

Recommended products
Cantaloupe
Mini Cucumbers
Bag of Organic Bananas
Lemon Hummus
Organic Pears & Rasberries Stage 2 Baby Food
Apples & Blueberries Stage 2 Baby Food
Organic Strawberries
Limes
Organic Lemonade
Organic Raspberries
Shredded Mozzarella
Coho Salmon
Organic Gluten Free Maple Buckwheat Flakes
Organic Green Seedless Grapes
Organic Tomato Basil Pasta Sauce
Large Lemon
Natural Chicken & Maple Breakfast Sausage Patty


In [13]:
#new user

X = {}
X['user_id'] = 225000
recommended_products = final(X)
print("Recommended products")
print("="*20)
for _,value in recommended_products[0]['top'].items():
    print(value)
print("Time taken :", recommended_products[1])

Recommended products
Banana
Bag of Organic Bananas
Organic Strawberries
Organic Baby Spinach
Organic Hass Avocado
Organic Avocado
Limes
Large Lemon
Strawberries
Organic Whole Milk
Time taken : 0:00:00.010808
